In [5]:
import os
import json

import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, BertConfig


In [6]:
model_path = r"D:\code\pretrain_model_dir\structbert-large"
# config = BertConfig.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

In [7]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [ ]:
token_output = tokenizer("你好世界", padding="max_length", truncation=True, max_length=16, return_tensors="pt")
input_ids = token_output["input_ids"]
token_type_ids = token_output["token_type_ids"]
attention_mask = token_output["attention_mask"]
print(input_ids.shape)

In [ ]:
model = BertModel.from_pretrained("bert-base-chinese")
model

In [ ]:
outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, output_hidden_states=True)
print(len(outputs))
print(outputs[0].shape)
print(outputs[1].shape)
# 有 13 个输出, 第一个是 embeddings, 后面 12 个是每一层的输出, bert-base-chinese 有 12 层
print(len(outputs[2]))
print(outputs[2][0].shape)
print(outputs[2][1].shape)
# 最后一层等价于 outputs[0]
assert torch.equal(outputs[0], outputs[2][12])

In [ ]:
outputs[0]

In [ ]:
outputs[2][12]

In [ ]:
torch.tensor(1).shape

In [ ]:
from data import *

In [ ]:
train_file = r"D:\code\py_nlp_classify\data\train.csv"
label_file = r"D:\code\py_nlp_classify\data\label.json"
train_df = load_dataframe(train_file)
label2id, _ = load_label(label_file)
train_dataset = CustomDataset(train_df, label2id, tokenizer)

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)

In [ ]:
next(iter(train_dataloader))[-1].shape